In [3]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [4]:
DATE_FORMAT = '%m.%Y'

In [5]:
def safe_mkdf(path, cols):
    dateparse = lambda x: pd.datetime.strptime(x, '%m.%Y')
    if os.path.exists(path):
        df = pd.read_csv(path, index_col=0, parse_dates = [1], date_parser=dateparse)
    else:
        df = pd.DataFrame(columns=cols)
    return df

In [11]:
class Hist_df():
    hist_columns = ['Date', 'Amount']
    
    
    def __init__(self, path):
        self.path = path
        self.df = safe_mkdf(path, Hist_df.hist_columns)
        
    def add_row(self, string_date, amnt):
        date_tm = pd.to_datetime(string_date, format=DATE_FORMAT)
        index = self.df.index.size
        self.df.loc[index] = pd.Series({Hist_df.hist_columns[0]: date_tm, Hist_df.hist_columns[1]: amnt})
        self.df.to_csv(self.path, date_format= DATE_FORMAT)
        
    def sort(self):
        self.df = self.df.sort_values(by = 'Date')
        self.df.index = np.arange(self.df.index.size)
        
    def get_grouped(self):
        return self.df.groupby(['Date']).sum()
        

In [7]:
def check_dates(hist_df):
    return not hist_df.df['Date'].duplicated().any()

In [126]:
class Investment():
    
    def safe_mkdir(self, path):
        if not os.path.exists(path):
            os.makedirs(path)
            
    def __init__(self, path):
        self.safe_mkdir(os.path.expanduser(path))
        
        self._stat_hist_filename = os.path.join(os.path.expanduser(path),'stat_hist.csv')
        
        self._deps_hist_filename = os.path.join(os.path.expanduser(path),'deps_hist.csv')
        
        self._withs_hist_filename = os.path.join(os.path.expanduser(path),'withs_hist.csv')
        
        self.stat_hist_df = Hist_df(self._stat_hist_filename)
        self.deps_hist_df = Hist_df(self._deps_hist_filename)
        self.withs_hist_df = Hist_df(self._withs_hist_filename)
        self.grouped_stat_diff_df = pd.DataFrame()
        self.grouped_withs_hist_df = pd.DataFrame()
        self.grouped_deps_hist_df = pd.DataFrame()
        
    def add_deposit(self, string_date, amnt):
        self.deps_hist_df.add_row(string_date, amnt)
        self.deps_hist_df.sort()
        self.grouped_deps_hist_df = self.deps_hist_df.get_grouped()
        
    def add_state(self, string_date, amnt):
        self.stat_hist_df.add_row(string_date, amnt)
        self.stat_hist_df.sort()
        self.grouped_stat_diff_df = self.stat_hist_df.df.set_index('Date').diff()
        if not check_dates(self.stat_hist_df):
            print("Warining! There is duplicates in state history")
            
    def add_withdrawal(self, string_date, amnt):
        self.withs_hist_df.add_row(string_date, amnt)
        self.withs_hist_df.sort()
        self.grouped_withs_hist_df = self.withs_hist_df.get_grouped()
        
    def build(self):
        self.stat_hist_df.sort()
        self.grouped_stat_diff_df = self.stat_hist_df.df.set_index('Date').diff()
        self.deps_hist_df.sort()
        self.grouped_deps_hist_df = self.deps_hist_df.get_grouped()
        self.withs_hist_df.sort()
        self.grouped_withs_hist_df = self.withs_hist_df.get_grouped()
        indexes = self.grouped_stat_diff_df.index.union(
            self.grouped_withs_hist_df.index).union(self.grouped_deps_hist_df.index)
        if len(indexes) > 0:
            first_i = indexes[0]
            last_i = indexes[-1]
            self.indexed_stat_diff_df = self.grouped_stat_diff_df.reindex(indexes, fill_value=0)
            self.indexed_withs_df = self.grouped_withs_hist_df.reindex(indexes, fill_value=0)
            self.indexed_deps_df = self.grouped_deps_hist_df.reindex(indexes, fill_value=0)
        
    def get_returns(self):
        return self.indexed_stat_diff_df + \
                self.indexed_withs_df - \
                self.indexed_deps_df

In [128]:
invest = Investment('./data')
invest.add_state('01.2017', 149)
invest.add_state('02.2017', 150)
invest.add_state('03.2017', 160)
invest.add_state('04.2017', 100)
invest.add_deposit('03.2017', 5)
invest.add_withdrawal('04.2017', 70)
invest.build()

In [133]:
print(invest.get_returns())

           Amount
Date             
2017-01-01    NaN
2017-02-01      1
2017-03-01      5
2017-04-01     10


In [38]:
r=pd.DataFrame(columns=['x', 'y'])
r.loc[0] = [1,2]
r

,x,y
0,1,2
